In [2]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### Building environment

- Build an agent to give us the best shower possible 
- Randomly temperature
- Goal: Maintain temperature between 37 and 39 degrees

In [34]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3) #Actions types val agent can do
        self.observation_space = Box(low=0, high=100, shape=(1,)) #Observation space
        self.state = 38 + random.randint(-3,3) #State = Current temperature
        self.shower_length = 60 #Shower time
    def step(self, action):
        self.state += action-1 #Apply impact of the action to the state
        self.shower_length -= 1 #Decrease shower time 

        #Calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1

        #Finish episode
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}

        return self.state, reward, done, info
    
    def render(self): #Implement visualization
        pass
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float) #Set initial temperature
        self.shower_length = 60 #Reset shower length
        return self.state

In [35]:
env = ShowerEnv()

In [25]:
env.observation_space.sample()

array([12.435839], dtype=float32)

In [26]:
env.action_space.sample()

0

### Template environment

In [15]:
episodes = 5 #Try the environment 5 times
for episode in range(1, episodes+1):
    state = env.reset() #Initial set of observations
    done = False
    score = 0

    while not done:
        env.render() #View the graphical representation of env
        action = env.action_space.sample() #Random selection of an action
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-60
Episode:2 Score:-4
Episode:3 Score:-60
Episode:4 Score:-40
Episode:5 Score:-38


### Train model

In [36]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/marcelo.hurtado/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [39]:
model.learn(total_timesteps=40000)

Logging to Training/Logs/PPO_9
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -19.3    |
| time/              |          |
|    fps             | 3847     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | -25.7      |
| time/                   |            |
|    fps                  | 2569       |
|    iterations           | 2          |
|    time_elapsed         | 1          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00784824 |
|    clip_fraction        | 0.0451     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.1       |
|    explained_variance   | -2.46e-05  |
|    learning_

In [40]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(12.0, 58.787753826796276)